<a href="https://colab.research.google.com/github/minjaecd123/study/blob/master/20250305_%EA%B8%B0%EC%98%A8%EC%9C%BC%EB%A1%9C%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#음식명과 설명을 활용하여

#음식이 추운 날, 온화한 날, 더운 날 중 어느 날씨에 적합한지 자동 분류

#TF-IDF (Term Frequency-Inverse Document Frequency) + 랜덤 포레스트 분류기(Random Forest Classifier)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd


food_df = pd.read_json('/content/drive/MyDrive/syon/Data/food_database.jsonl', lines=True, encoding='utf-8')
food_df

,CKG_NM,CKG_STA_ACTO_NM,CKG_IPDC,CKG_GROUP
0,No Data,술안주,냉장고를 부탁해 이연복 복꽃엔딩,안주
1,명란알게맛살돌솥비빔밥,영양식,게맛살과 명란젓을 이용해서 알밥 만들기 해 볼께요.,건강식
2,소고기무국,영양식,오늘이 초복이라고 하네요. 개인의 취향에 따라 더위를 이겨내기 위한 음식들을 많이...,건강식
3,야채볼,초스피드,불을 사용하지 않고 간편하게 먹을 수 있는 한그릇 요리 레시피입니다.,간편식
4,쭈꾸미숙회,술안주,낚시를 좋아하는 남편의 지인 덕분에 살아있는 주꾸미와 문어를 맛보았습니다. 잘못 ...,안주
...,...,...,...,...
39706,�遊摹�,손님접대,옛부터 여자는 꽃으로도 때리지말라고 했죠 꽃으로도..꼬..츠로... 꼬..추로.....,집밥
39707,��가슴살라따뚜이,일상,건강한 맛 가득 멋진 토마토소스와 야채들의 조화 그곳에 닭가슴살이 있다면 말이 필요...,집밥
39708,��경채소고기볶음,영양식,아삭한 청경채와 소고기 넣고 굴소스로 감칠맛을 낸 간단반찬 청경채 소고기볶음입니다^...,건강식
39709,����면,초스피드,#밀키트 #����멘만들기 #����멘밀키트 #대만식비빔면 #����면 #넙적면 ...,간편식


In [4]:
food_df = food_df.drop(columns=['CKG_STA_ACTO_NM'])

In [5]:
food_df

,CKG_NM,CKG_IPDC,CKG_GROUP
0,No Data,냉장고를 부탁해 이연복 복꽃엔딩,안주
1,명란알게맛살돌솥비빔밥,게맛살과 명란젓을 이용해서 알밥 만들기 해 볼께요.,건강식
2,소고기무국,오늘이 초복이라고 하네요. 개인의 취향에 따라 더위를 이겨내기 위한 음식들을 많이...,건강식
3,야채볼,불을 사용하지 않고 간편하게 먹을 수 있는 한그릇 요리 레시피입니다.,간편식
4,쭈꾸미숙회,낚시를 좋아하는 남편의 지인 덕분에 살아있는 주꾸미와 문어를 맛보았습니다. 잘못 ...,안주
...,...,...,...
39706,�遊摹�,옛부터 여자는 꽃으로도 때리지말라고 했죠 꽃으로도..꼬..츠로... 꼬..추로.....,집밥
39707,��가슴살라따뚜이,건강한 맛 가득 멋진 토마토소스와 야채들의 조화 그곳에 닭가슴살이 있다면 말이 필요...,집밥
39708,��경채소고기볶음,아삭한 청경채와 소고기 넣고 굴소스로 감칠맛을 낸 간단반찬 청경채 소고기볶음입니다^...,건강식
39709,����면,#밀키트 #����멘만들기 #����멘밀키트 #대만식비빔면 #����면 #넙적면 ...,간편식


In [6]:
food_df = food_df.rename(columns={'CKG_NM': '음식명', 'CKG_IPDC': '설명'})
food_df

,음식명,설명,CKG_GROUP
0,No Data,냉장고를 부탁해 이연복 복꽃엔딩,안주
1,명란알게맛살돌솥비빔밥,게맛살과 명란젓을 이용해서 알밥 만들기 해 볼께요.,건강식
2,소고기무국,오늘이 초복이라고 하네요. 개인의 취향에 따라 더위를 이겨내기 위한 음식들을 많이...,건강식
3,야채볼,불을 사용하지 않고 간편하게 먹을 수 있는 한그릇 요리 레시피입니다.,간편식
4,쭈꾸미숙회,낚시를 좋아하는 남편의 지인 덕분에 살아있는 주꾸미와 문어를 맛보았습니다. 잘못 ...,안주
...,...,...,...
39706,�遊摹�,옛부터 여자는 꽃으로도 때리지말라고 했죠 꽃으로도..꼬..츠로... 꼬..추로.....,집밥
39707,��가슴살라따뚜이,건강한 맛 가득 멋진 토마토소스와 야채들의 조화 그곳에 닭가슴살이 있다면 말이 필요...,집밥
39708,��경채소고기볶음,아삭한 청경채와 소고기 넣고 굴소스로 감칠맛을 낸 간단반찬 청경채 소고기볶음입니다^...,건강식
39709,����면,#밀키트 #����멘만들기 #����멘밀키트 #대만식비빔면 #����면 #넙적면 ...,간편식


#대략적인 표본 만들기

#ex) 김치찌개 -> "찌개" -> 추운날 먹는 음식

In [7]:
food_df.to_csv("labeled_foods.csv", index=False, encoding="utf-8-sig")

df = pd.read_csv("labeled_foods.csv", encoding="utf-8-sig")

In [8]:
# 새로운 'label' 컬럼 추가 (초기값은 '미정'으로 설정)
df['label'] = '미정'
df['설명'] = df['설명'].astype(str).fillna('')
df['음식명'] = df['음식명'].astype(str).fillna('')

# 특정 음식에 대해 수동 라벨링
df.loc[df['설명'] == '김치찌개', 'label'] = '추운 날'
df.loc[df['설명'] == '비빔밥', 'label'] = '온화한 날'
df.loc[df['설명'] == '냉면', 'label'] = '더운 날'


In [9]:
import pandas as pd

# NaN 값이 있는 경우 대비하여 df['설명']과 df['음식명']을 문자열로 변환
df['설명'] = df['설명'].astype(str).fillna('')
df['음식명'] = df['음식명'].astype(str).fillna('')

# 자동 라벨링 함수 (음식명 + 설명 모두 고려)
def auto_labeling(row):
    # 음식명과 설명을 결합하여 분석
    text = row['음식명'] + " " + row['설명']

    cold_keywords = ['국', '찌개', '전골', '국밥', '탕', '설렁탕', '곰탕', '육개장', '감자탕', '삼계탕', '곱창전골', '부대찌개', '홍합탕', '우거지국', '갈비찜', '보쌈', '족발', '꼬리곰탕', '팥죽', '떡국', '호빵', '어묵탕', '고구마', '매운탕', '추어탕', '닭도리탕', '사골국', '우럭탕', '명태국', '토종닭백숙', '호떡', '군밤', '튀김우동', '치즈퐁듀', '양념갈비', '바비큐립', '땅콩호박스프', '붕어빵', '군고구마', '생강차']

    warm_keywords = ['나물', '볶음', '국수', '파스타', '토스트', '절임', '봄나물', '구이', '떡', '케이크', '케익', '샌드위치', '된장찌개', '버섯전골', '비빔밥', '불고기', '찜닭', '연근조림', '시금치나물', '도토리묵', '생선구이', '잡채', '버섯볶음', '연어스테이크', '오리구이', '차돌박이샐러드', '무생채', '오곡밥', '고구마그라탕', '버터구이감자', '사과파이']

    hot_keywords = ['보신','냉', '회', '샐러드', '초밥', '물회', '냉국', '빙수', '아이스크림', '초복', '중복', '말복', '열무', '보양', '수박', '빙', '주스', '비빔', '오이', '열무', '냉면', '막국수', '콩국수', '오이냉국', '동치미', '묵사발', '해파리냉채', '냉채족발', '과일화채', '오미자차', '레모네이드', '참외화채', '토마토냉채', '생과일주스', '새우칵테일', '게살냉채', '초계탕', '메밀소바', '냉파스타']

    if any(word in text for word in cold_keywords):
        return '추운 날'
    elif any(word in text for word in warm_keywords):
        return '온화한 날'
    elif any(word in text for word in hot_keywords):
        return '더운 날'
    else:
        return '미정'

# '미정'인 데이터 자동 분류 (apply()에 axis=1 추가)
df.loc[df['label'] == '미정', 'label'] = df[df['label'] == '미정'].apply(auto_labeling, axis=1)

# ✅ 결과 확인 (로컬 실행 시)
df.head()



,음식명,설명,CKG_GROUP,label
0,No Data,냉장고를 부탁해 이연복 복꽃엔딩,안주,더운 날
1,명란알게맛살돌솥비빔밥,게맛살과 명란젓을 이용해서 알밥 만들기 해 볼께요.,건강식,온화한 날
2,소고기무국,오늘이 초복이라고 하네요. 개인의 취향에 따라 더위를 이겨내기 위한 음식들을 많이...,건강식,추운 날
3,야채볼,불을 사용하지 않고 간편하게 먹을 수 있는 한그릇 요리 레시피입니다.,간편식,미정
4,쭈꾸미숙회,낚시를 좋아하는 남편의 지인 덕분에 살아있는 주꾸미와 문어를 맛보았습니다. 잘못 ...,안주,더운 날


In [10]:
df['label'].value_counts()

,count
label,
미정,14618
온화한 날,11560
추운 날,9628
더운 날,3905


In [11]:
df.to_csv("labeled_foods_result.csv", index=False, encoding="utf-8-sig") #결과확인해보기

#대략적으로 만들어진 표본을 바탕으로 아직 나누지 못한 미정 카테고리 분류하기

# TF-IDF (Term Frequency-Inverse Document Frequency) + 랜덤 포레스트 분류기

In [12]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score

# 데이터 로드
data_path = "labeled_foods_result.csv"  # 파일 경로 지정
df = pd.read_csv(data_path)

# 1. 라벨이 '미정'이 아닌 데이터만 학습에 사용
df_labeled = df[df['label'] != '미정']
df_unlabeled = df[df['label'] == '미정']

# 2. TF-IDF 벡터화
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df_labeled['설명'].astype(str))
y = df_labeled['label']

# 3. 데이터 불균형 확인
label_counts = y.value_counts()
print("라벨별 데이터 개수:\n", label_counts)

# 4. 데이터 분할 (학습 80%, 테스트 20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 5. 클래스 가중치 계산 (데이터 불균형 보정)
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y), y=y)
class_weight_dict = {cls: weight for cls, weight in zip(np.unique(y), class_weights)}

# 6. 랜덤 포레스트 분류 모델 학습 (가중치 적용)
clf = RandomForestClassifier(n_estimators=100, random_state=42, class_weight=class_weight_dict)
clf.fit(X_train, y_train)

# 7. 모델 성능 평가
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"모델 정확도: {accuracy:.4f}")

# 8. '미정' 데이터가 존재하는 경우에만 예측 수행
if not df_unlabeled.empty:
    X_unlabeled = vectorizer.transform(df_unlabeled['설명'].astype(str))
    df_unlabeled['predicted_label'] = clf.predict(X_unlabeled)

    # 9. 기존 데이터에서 '미정' 라벨을 예측된 라벨로 업데이트
    df.loc[df['label'] == '미정', 'label'] = df_unlabeled['predicted_label'].values
    print("'미정' 데이터 라벨링 완료!")
else:
    print("'미정' 라벨이 없는 상태이므로 예측을 수행하지 않습니다.")

# 10. 최종 데이터 저장 및 확인
df.to_csv("updated_food_labels.csv", index=False)
print("업데이트된 데이터 저장 완료: updated_food_labels.csv")

라벨별 데이터 개수:
 label
온화한 날    11560
추운 날      9628
더운 날      3905
Name: count, dtype: int64
모델 정확도: 0.7205


<ipython-input-12-6c09a3f446bd>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unlabeled['predicted_label'] = clf.predict(X_unlabeled)


'미정' 데이터 라벨링 완료!
업데이트된 데이터 저장 완료: updated_food_labels.csv


In [13]:
df['label'].value_counts()

,count
label,
온화한 날,24047
추운 날,11440
더운 날,4224


In [14]:
# 11. 기온에 따른 음식 추천 함수
import random
def recommend_food(temperature):
    if temperature >= 25:
        label = "더운 날"
    elif temperature <= 10:
        label = "추운 날"
    else:
        label = "온화한 날"

    filtered_df = df[df['label'] == label]
    if not filtered_df.empty:
        return random.choice(filtered_df['음식명'].tolist())
    else:
        return "추천할 음식이 없습니다."

# 예시 실행 (기온 입력 시 음식 추천)
temp = 15  # 예시 기온
generated_food = recommend_food(temp)
print(f"추천 음식: {generated_food}")

추천 음식: 도토리묵샐러드


# 기상청 api

In [15]:
import requests
import datetime

def get_short_term_weather(api_key, nx, ny, days_ahead):
    """
    단기예보 API를 사용하여 지정된 날짜 후의 기온, 강수량, 습도, 날씨 데이터를 가져옵니다.
    """
    target_date = (datetime.datetime.now() + datetime.timedelta(days=days_ahead)).strftime("%Y%m%d")
    base_time = "0500"  # 5시 기준 예보 데이터
    url = "http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst"
    params = {
        "serviceKey": api_key,
        "numOfRows": 100,
        "pageNo": 1,
        "dataType": "JSON",
        "base_date": target_date,
        "base_time": base_time,
        "nx": nx,
        "ny": ny
    }
    response = requests.get(url, params=params)
    data = response.json()
    return extract_weather_summary(data, f"{days_ahead}일 후")

def extract_weather_summary(data, label):
    """
    API 응답 데이터를 가독성 있게 정리하여 반환합니다.
    """
    if "response" not in data or "body" not in data["response"] or "items" not in data["response"]["body"]:
        return f"{label} 날씨 데이터 없음"

    items = data["response"]["body"]["items"]["item"]
    weather_summary = {
        "기온": None,
        "강수량": None,
        "습도": None,
        "날씨": None,
        "날짜": None,
        "날씨 상태": None  # 추가된 변수
    }

    for item in items:
        category = item["category"]
        if category == "TMP":
            temperature = float(item["fcstValue"])
            weather_summary["기온"] = temperature
            weather_summary["날짜"] = item["fcstDate"]
            # 기온에 따른 날씨 상태 설정
            if temperature >= 25:
                weather_summary["날씨 상태"] = "더운 날"
            elif temperature <= 10:
                weather_summary["날씨 상태"] = "추운 날"
            else:
                weather_summary["날씨 상태"] = "온화한 날"
        elif category == "PCP":
            weather_summary["강수량"] = item["fcstValue"]
        elif category == "REH":
            weather_summary["습도"] = item["fcstValue"]
        elif category == "SKY":
            weather_summary["날씨"] = "맑음" if item["fcstValue"] == "1" else "흐림"

    return (f"{label} ({weather_summary['날짜']}):\n"
            f"기온: {weather_summary['기온']}°C\n"
            f"강수량: {weather_summary['강수량']}\n"
            f"습도: {weather_summary['습도']}%\n"
            f"날씨: {weather_summary['날씨']}\n"
            f"날씨 상태: {weather_summary['날씨 상태']}\n")

# 예제 실행 (API 키와 좌표 값을 입력해야 합니다)
API_KEY = "9/J+JAxSAQW37NA9zqY7YcAMkvZuUyZpDaGEfASZzlJFQQndFFdbWDaCZ4aiMWBp4NwNZOI2bMPk5H3NU0aq/g=="  # 본인의 API 키 입력
NX, NY = 60, 127  # 서울 기준 예제 좌표 (사용자의 위치에 맞게 변경)
REG_ID = "11B10101"  # 서울/경기 지역 코드 예제

# 현재 날씨 데이터 가져오기
short_term_weather = get_short_term_weather(API_KEY, NX, NY, 0)
print(short_term_weather)

0일 후 (20250310):
기온: 14.0°C
강수량: 강수없음
습도: 40%
날씨: 맑음
날씨 상태: 온화한 날



#기상청 api 와 추천 프로그램 결합

In [16]:
import random

def recommend_food_by_weather(data, label, df):
    """
    날씨 데이터에 따라 음식을 추천하는 함수
    """
    weather_info = extract_weather_summary(data, label)

    # 날씨 상태 추출
    weather_status_line = [line for line in weather_info.split("\n") if "날씨 상태" in line]

    if not weather_status_line:
        return "날씨 정보를 가져올 수 없습니다."

    weather_status = weather_status_line[0].split(":")[-1].strip()  # "더운 날", "추운 날", "온화한 날" 추출

    # 해당 날씨 상태에 맞는 음식 추천
    filtered_df = df[df['label'] == weather_status]
    if not filtered_df.empty:
        return random.choice(filtered_df['음식명'].tolist())
    else:
        return "추천할 음식이 없습니다."

# 예시 데이터프레임 (실제 코드에서는 기존 df를 사용)
import pandas as pd

# 예시 날씨 데이터
sample_weather_data = {
    "response": {
        "body": {
            "items": {
                "item": [
                    {"category": "TMP", "fcstValue": "15", "fcstDate": "2025-03-06"},
                    {"category": "PCP", "fcstValue": "0mm"},
                    {"category": "REH", "fcstValue": "55"},
                    {"category": "SKY", "fcstValue": "1"}
                ]
            }
        }
    }
}

# 추천 음식 실행
recommended_food = recommend_food_by_weather(sample_weather_data, "서울", df)
print(f"추천 음식: {recommended_food}")


추천 음식: 맛살달걀범벅
